In [3]:
import time, logging
from pathlib import Path
from flofish.experiment import Experiment
from flofish.image import Image

In [6]:
import os
os.getcwd()



'/Users/adele/PycharmProjects/flofish/flofish'

### Download test data

In [12]:
from napari_flofish import get_smRNAfish_Ecoli_rpoD_rnlAB_hipBA
get_smRNAfish_Ecoli_rpoD_rnlAB_hipBA(Path("tests/data/exp16/expected"), cleanup=False)

### Read in microscope images

In [13]:
exp = Experiment.from_cfg_file("tests/data/exp16/input/config.json")
exp.create_image_list()

2025-02-25 16:53:50,946	[INFO]                                   from..._file()	 line  30	found cfg file tests/data/exp16/input/config.json
2025-02-25 16:53:50,947	[INFO]                                   crea..._list()	 line  43	getting .vsi files from flofish/tests/data/exp16/input


In [15]:
for params in exp.images.values():
    logging.info(params)
    my_image = Image.from_dict(params, exp)

    tic = time.time()
    my_image.read_image()
    my_image.read_cells()
    my_image.align()
    my_image.create_grgb()

    # save image (write to dir)
    my_image.save_input_layers()
    my_image.time['01-configure'] = time.time() - tic
    my_image.save_metadata("configure")

### Segment images

### Detect spots